In [1]:
from catboost import CatBoost, CatBoostClassifier
import lightgbm as lgb

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn import ensemble,metrics,cross_validation,preprocessing,grid_search
path='C:/Projects/Hackerearth_2/'

C:\Users\sharm_000\Anaconda2\envs\py36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data=pd.read_csv(path+'train.csv',dtype={'roadId':object})
labels=pd.read_csv(path+'labels.csv',dtype={'roadId':object})
test_data=pd.read_csv(path+'test.csv',dtype={'roadId':object})
print (data.shape,labels.shape)
len(np.unique(data['roadId'])),len(np.unique(test_data['roadId']))

(28914, 14) (4825, 3)


(4825, 400)

In [4]:
new_data=pd.merge(data,labels,on='roadId',how='left')
new_data.columns=[i.strip(' ')for i in new_data.columns]
# new_data['roadCoordinates_count']=new_data['roadCoordinates'].apply(lambda x: len(x.split()))
new_data['laneLineCoordinates_count']=new_data['laneLineCoordinates'].apply(lambda x: len(x.split()))
new_data=new_data.fillna(0)
new_data.head(1)

,roadId,totalLaneLines,laneLineId,laneLineCoordinates,totalLinesOnLeft,totalLaneLinesOnRight,distFromLaneLineOnLeft,distFromLaneLineOnRight,laneLineLength,roadLength,noOfIntersectingLaneLinesLeft,noOfIntersectingLaneLinesRight,isIntersectingWithRoadGeometry,roadCategory,roadCoordinates,noOfLanes,laneLineCoordinates_count
0,3177248677,3,X3177248677,11.117636626741222 49.43799310961545|11.11761...,2,0,3.863433276,,4.396337,40.57071,0,0,false,2,11.11747 49.43763|11.11753 49.43776|11.11761 ...,2,3


In [5]:
def num_con(x):
    try:
        return float(x)
    except:
        return 0
    
def geom(x):
    if x=='false':
        return 0
    else:
        return 1

In [6]:
new_data['distFromLaneLineOnLeft']=new_data['distFromLaneLineOnLeft'].apply(lambda x: num_con(x))
new_data['distFromLaneLineOnRight']=new_data['distFromLaneLineOnRight'].apply(lambda x: num_con(x))
new_data['isIntersectingWithRoadGeometry']=new_data['isIntersectingWithRoadGeometry'].apply(lambda x: num_con(x))

In [7]:
new_data.columns

Index(['roadId', 'totalLaneLines', 'laneLineId', 'laneLineCoordinates',
       'totalLinesOnLeft', 'totalLaneLinesOnRight', 'distFromLaneLineOnLeft',
       'distFromLaneLineOnRight', 'laneLineLength', 'roadLength',
       'noOfIntersectingLaneLinesLeft', 'noOfIntersectingLaneLinesRight',
       'isIntersectingWithRoadGeometry', 'roadCategory', 'roadCoordinates',
       'noOfLanes', 'laneLineCoordinates_count'],
      dtype='object')

In [8]:
train_pro_data=new_data.groupby(by='roadId').agg({'laneLineId':{'laneLineId':'nunique'},
        'totalLaneLines':{'totalLaneLines_sum':'sum','totalLaneLines_mean':'mean','totalLaneLines_min':'min','totalLaneLines_max':'max'},\
        'distFromLaneLineOnRight':{'distFromLaneLineOnRight_mean':'mean','distFromLaneLineOnRight_min':'min',\
        'distFromLaneLineOnRight_sum':'sum',  'distFromLaneLineOnRight_max':'max'},\
        'distFromLaneLineOnLeft':{'distFromLaneLineOnLeft_mean':'mean','distFromLaneLineOnLeft_min':'min',\
        'distFromLaneLineOnLeft_sum':'sum',                   'distFromLaneLineOnLeft_max':'max'},\
        'laneLineLength':{'laneLineLength_mean':'mean','laneLineLength_min':'min','laneLineLength_max':'max','laneLineLength_sum':'sum',},\
        'roadLength':{'roadLength_mean':'mean','roadLength_min':'min','roadLength_max':'max'},\
        'noOfIntersectingLaneLinesLeft':{'noOfIntersectingLaneLinesLeft_sum':'sum',\
        'noOfIntersectingLaneLinesLeft_min':'min','noOfIntersectingLaneLinesLeft_max':'max',\
        'noOfIntersectingLaneLinesLeft_mean':'mean'},\
        'noOfIntersectingLaneLinesRight':{'noOfIntersectingLaneLinesRight_sum':'sum',\
                                          'noOfIntersectingLaneLinesRight_min':'min',\
                                          'noOfIntersectingLaneLinesRight_max':'max'},\
        'roadCategory':{'roadCategory':'mean'},'isIntersectingWithRoadGeometry':{'isIntersectingWithRoadGeometry':'sum'},\
        'laneLineCoordinates_count':{'laneLineCoordinates_count':'mean','laneLineCoordinates_min':'min','laneLineCoordinates_max':'max',\
         'laneLineCoordinates_sum':'sum'},'noOfLanes':{'noOfLanes':'mean'}}).reset_index()

train_pro_data.columns=train_pro_data.columns.droplevel(0)
train_pro_data.rename(columns={'':'roadId'},inplace=True)

In [9]:
train_pro_data.head()

,roadId,laneLineId,totalLaneLines_sum,totalLaneLines_mean,totalLaneLines_min,totalLaneLines_max,distFromLaneLineOnRight_mean,distFromLaneLineOnRight_min,distFromLaneLineOnRight_sum,distFromLaneLineOnRight_max,...,noOfIntersectingLaneLinesRight_sum,noOfIntersectingLaneLinesRight_min,noOfIntersectingLaneLinesRight_max,roadCategory,isIntersectingWithRoadGeometry,laneLineCoordinates_count,laneLineCoordinates_min,laneLineCoordinates_max,laneLineCoordinates_sum,noOfLanes
0,1003085808,5,25,5.0,5,5,2.118718,0.0,10.593588,3.653125,...,0,0,0,2,0,8.600000,4,14,43,2
1,1003388168,3,9,3.0,3,3,2.913640,0.0,8.740921,4.690038,...,0,0,0,2,0,6.333333,5,9,19,2
2,1003388169,4,16,4.0,4,4,3.167543,0.0,12.670172,4.901373,...,0,0,0,2,0,4.750000,3,7,19,2
3,1009949115,4,16,4.0,4,4,2.858797,0.0,11.435189,3.888012,...,0,0,0,1,0,13.500000,10,16,54,3
4,1009975629,4,16,4.0,4,4,2.939509,0.0,11.758036,3.952289,...,0,0,0,1,0,5.250000,4,6,21,3


In [10]:
train_pro_data.columns

Index(['roadId', 'laneLineId', 'totalLaneLines_sum', 'totalLaneLines_mean',
       'totalLaneLines_min', 'totalLaneLines_max',
       'distFromLaneLineOnRight_mean', 'distFromLaneLineOnRight_min',
       'distFromLaneLineOnRight_sum', 'distFromLaneLineOnRight_max',
       'distFromLaneLineOnLeft_mean', 'distFromLaneLineOnLeft_min',
       'distFromLaneLineOnLeft_sum', 'distFromLaneLineOnLeft_max',
       'laneLineLength_mean', 'laneLineLength_min', 'laneLineLength_max',
       'laneLineLength_sum', 'roadLength_mean', 'roadLength_min',
       'roadLength_max', 'noOfIntersectingLaneLinesLeft_sum',
       'noOfIntersectingLaneLinesLeft_min',
       'noOfIntersectingLaneLinesLeft_max',
       'noOfIntersectingLaneLinesLeft_mean',
       'noOfIntersectingLaneLinesRight_sum',
       'noOfIntersectingLaneLinesRight_min',
       'noOfIntersectingLaneLinesRight_max', 'roadCategory',
       'isIntersectingWithRoadGeometry', 'laneLineCoordinates_count',
       'laneLineCoordinates_min', 'laneLine

In [11]:
train_pro_data['areaFromLaneLine_mean']=train_pro_data['distFromLaneLineOnRight_mean']*train_pro_data['distFromLaneLineOnLeft_mean']
train_pro_data['areaFromLaneLine_min']=train_pro_data['distFromLaneLineOnRight_min']*train_pro_data['distFromLaneLineOnLeft_min']
train_pro_data['areaFromLaneLine_sum']=train_pro_data['distFromLaneLineOnRight_sum']*train_pro_data['distFromLaneLineOnLeft_sum']
train_pro_data['areaFromLaneLine_max']=train_pro_data['distFromLaneLineOnRight_max']*train_pro_data['distFromLaneLineOnLeft_max']

In [12]:
train_pro_data['ratio_Intersect_sum']=train_pro_data['noOfIntersectingLaneLinesLeft_sum']/train_pro_data['noOfIntersectingLaneLinesRight_sum']
train_pro_data['ratio_Intersect_min']=train_pro_data['noOfIntersectingLaneLinesLeft_min']/train_pro_data['noOfIntersectingLaneLinesRight_min']
train_pro_data['ratio_Intersect_max']=train_pro_data['noOfIntersectingLaneLinesLeft_max']/train_pro_data['noOfIntersectingLaneLinesRight_max']

In [13]:
train_pro_data['noOfLanes']=train_pro_data['noOfLanes'].apply(lambda x:str(x))

In [14]:
lbl=preprocessing.LabelEncoder()
train_pro_data['noOfLanes']=lbl.fit_transform(train_pro_data['noOfLanes'])

In [15]:
# train_pro_data=pd.concat([train_pro_data,pd.get_dummies(train_pro_data['roadCategory'],prefix='rC',prefix_sep='_')[['rC_1','rC_2']]],axis=1)

In [16]:
# del train_pro_data['roadCategory']
# train_pro_data.shape

In [17]:
# new_test_data=pd.merge(test_data,labels,on='roadId',how='left')
test_data.columns=[i.strip(' ')for i in test_data.columns]
test_data.shape,len(np.unique(test_data['roadId']))

((2493, 14), 400)

In [18]:
test_data.head()

,roadId,totalLaneLines,laneLineId,laneLineCoordinates,totalLinesOnLeft,totalLaneLinesOnRight,distFromLaneLineOnLeft,distFromLaneLineOnRight,laneLineLength,roadLength,noOfIntersectingLaneLinesLeft,noOfIntersectingLaneLinesRight,isIntersectingWithRoadGeometry,roadCategory
0,291531296,5,X291531296,11.329899604343693 43.72956824243773|11.33004...,2,2,5.655398294,3.49899823,12.054597,12.056004,0,0,false,1
1,291531296,5,1291531296,11.329900151795446 43.729503478051974|11.3300...,4,0,3.685959755,,12.060141,12.056004,0,0,false,1
2,291531296,5,2291531296,11.329899913660833 43.72953696742431|11.33004...,3,1,3.49899823,3.685959755,12.052468,12.056004,0,0,false,1
3,291531296,5,3291531296,11.329899175813209 43.72961906004278|11.33004...,1,3,3.481580911,5.655398294,12.046642,12.056004,0,0,false,1
4,291531296,5,4291531296,11.329899073264464 43.729650480155186|11.3300...,0,4,,3.481580911,12.054345,12.056004,0,0,false,1


In [19]:
# new_test_data['roadCoordinates_count']=new_test_data['roadCoordinates'].apply(lambda x: len(x.split()))
test_data['laneLineCoordinates_count']=test_data['laneLineCoordinates'].apply(lambda x: len(x.split()))
test_data=test_data.fillna(0)
print (test_data.shape)
test_data.head(2)

(2493, 15)


,roadId,totalLaneLines,laneLineId,laneLineCoordinates,totalLinesOnLeft,totalLaneLinesOnRight,distFromLaneLineOnLeft,distFromLaneLineOnRight,laneLineLength,roadLength,noOfIntersectingLaneLinesLeft,noOfIntersectingLaneLinesRight,isIntersectingWithRoadGeometry,roadCategory,laneLineCoordinates_count
0,291531296,5,X291531296,11.329899604343693 43.72956824243773|11.33004...,2,2,5.655398294,3.49899823,12.054597,12.056004,0,0,false,1,3
1,291531296,5,1291531296,11.329900151795446 43.729503478051974|11.3300...,4,0,3.685959755,,12.060141,12.056004,0,0,false,1,3


In [20]:
test_data['distFromLaneLineOnLeft']=test_data['distFromLaneLineOnLeft'].apply(lambda x: num_con(x))
test_data['distFromLaneLineOnRight']=test_data['distFromLaneLineOnRight'].apply(lambda x: num_con(x))
test_data['isIntersectingWithRoadGeometry']=test_data['isIntersectingWithRoadGeometry'].apply(lambda x: num_con(x))

In [21]:
test_pro_data=test_data.groupby(by='roadId').agg({'laneLineId':{'laneLineId':'nunique'},
        'totalLaneLines':{'totalLaneLines_sum':'sum','totalLaneLines_mean':'mean','totalLaneLines_min':'min','totalLaneLines_max':'max'},\
        'distFromLaneLineOnRight':{'distFromLaneLineOnRight_mean':'mean','distFromLaneLineOnRight_min':'min',\
        'distFromLaneLineOnRight_sum':'sum',  'distFromLaneLineOnRight_max':'max'},\
        'distFromLaneLineOnLeft':{'distFromLaneLineOnLeft_mean':'mean','distFromLaneLineOnLeft_min':'min',\
        'distFromLaneLineOnLeft_sum':'sum',                   'distFromLaneLineOnLeft_max':'max'},\
        'laneLineLength':{'laneLineLength_mean':'mean','laneLineLength_min':'min','laneLineLength_max':'max','laneLineLength_sum':'sum',},\
        'roadLength':{'roadLength_mean':'mean','roadLength_min':'min','roadLength_max':'max'},\
        'noOfIntersectingLaneLinesLeft':{'noOfIntersectingLaneLinesLeft_sum':'sum',\
        'noOfIntersectingLaneLinesLeft_min':'min','noOfIntersectingLaneLinesLeft_max':'max',\
        'noOfIntersectingLaneLinesLeft_mean':'mean'},\
        'noOfIntersectingLaneLinesRight':{'noOfIntersectingLaneLinesRight_sum':'sum',\
                                          'noOfIntersectingLaneLinesRight_min':'min',\
                                          'noOfIntersectingLaneLinesRight_max':'max'},\
        'roadCategory':{'roadCategory':'mean'},'isIntersectingWithRoadGeometry':{'isIntersectingWithRoadGeometry':'sum'},\
        'laneLineCoordinates_count':{'laneLineCoordinates_count':'mean','laneLineCoordinates_min':'min','laneLineCoordinates_max':'max',\
         'laneLineCoordinates_sum':'sum'}}).reset_index()

test_pro_data.columns=test_pro_data.columns.droplevel(0)
test_pro_data.rename(columns={'':'roadId'},inplace=True)

In [22]:
test_pro_data['areaFromLaneLine_mean']=test_pro_data['distFromLaneLineOnRight_mean']*test_pro_data['distFromLaneLineOnLeft_mean']
test_pro_data['areaFromLaneLine_min']=test_pro_data['distFromLaneLineOnRight_min']*test_pro_data['distFromLaneLineOnLeft_min']
test_pro_data['areaFromLaneLine_sum']=test_pro_data['distFromLaneLineOnRight_sum']*test_pro_data['distFromLaneLineOnLeft_sum']
test_pro_data['areaFromLaneLine_max']=test_pro_data['distFromLaneLineOnRight_max']*test_pro_data['distFromLaneLineOnLeft_max']

In [23]:
test_pro_data['ratio_Intersect_sum']=test_pro_data['noOfIntersectingLaneLinesLeft_sum']/test_pro_data['noOfIntersectingLaneLinesRight_sum']
test_pro_data['ratio_Intersect_min']=test_pro_data['noOfIntersectingLaneLinesLeft_min']/test_pro_data['noOfIntersectingLaneLinesRight_min']
test_pro_data['ratio_Intersect_max']=test_pro_data['noOfIntersectingLaneLinesLeft_max']/test_pro_data['noOfIntersectingLaneLinesRight_max']

In [24]:
test_pro_data.columns

Index(['roadId', 'laneLineId', 'totalLaneLines_sum', 'totalLaneLines_mean',
       'totalLaneLines_min', 'totalLaneLines_max',
       'distFromLaneLineOnRight_mean', 'distFromLaneLineOnRight_min',
       'distFromLaneLineOnRight_sum', 'distFromLaneLineOnRight_max',
       'distFromLaneLineOnLeft_mean', 'distFromLaneLineOnLeft_min',
       'distFromLaneLineOnLeft_sum', 'distFromLaneLineOnLeft_max',
       'laneLineLength_mean', 'laneLineLength_min', 'laneLineLength_max',
       'laneLineLength_sum', 'roadLength_mean', 'roadLength_min',
       'roadLength_max', 'noOfIntersectingLaneLinesLeft_sum',
       'noOfIntersectingLaneLinesLeft_min',
       'noOfIntersectingLaneLinesLeft_max',
       'noOfIntersectingLaneLinesLeft_mean',
       'noOfIntersectingLaneLinesRight_sum',
       'noOfIntersectingLaneLinesRight_min',
       'noOfIntersectingLaneLinesRight_max', 'roadCategory',
       'isIntersectingWithRoadGeometry', 'laneLineCoordinates_count',
       'laneLineCoordinates_min', 'laneLine

In [25]:
X=test_pro_data.columns.tolist()[1:]
Y='noOfLanes'
len(X)

40

In [26]:
train_pro_data=train_pro_data.fillna(0)

In [28]:
train_X,test_X,train_y,test_y=cross_validation.train_test_split(train_pro_data[X],train_pro_data[Y],test_size=.3,random_state =74)
train_X.shape,train_y.shape,test_X.shape,test_y.shape

((3377, 40), (3377,), (1448, 40), (1448,))

### CAT

In [30]:
random_seed=85
clf = CatBoostClassifier(learning_rate=0.01, iterations=200, random_seed=random_seed, \
                          loss_function='MultiClass', class_count=6)
optimized_CT = grid_search.GridSearchCV(clf,{'depth':[4,],'rsm':[0.1,.3,.6,.8]
                    }, verbose=1) 

optimized_CT.fit(train_pro_data[X],train_pro_data[Y])

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  1.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=<catboost.core.CatBoostClassifier object at 0x00000280138360F0>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'depth': [4], 'rsm': [0.1, 0.3, 0.6, 0.8]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [ ]:
metrics.f1_score(train_pro_data[Y],np.ravel(optimized_CT.predict(train_pro_data[X])),average='weighted')

In [ ]:

# clf.fit(train_pro_data[X], train_pro_data[Y])
pred_cat=np.ravel(optimized_CT.predict(test_pro_data[X]))

print (np.unique(pred_cat,return_counts=True))

pred_cat=lbl.inverse_transform(pred_cat.astype(int))
print(np.unique(pred_cat))

sub_samp2=pd.DataFrame(test_pro_data['roadId'])
sub_samp2['noOfLanes']=pred_cat

sub_samp2.to_csv(path+str(random_seed)+'1cat_model_less_feature.csv',index=False)

In [ ]:
t4_params = {
    'boosting_type': 'gbdt', 'objective': 'multiclass', 'nthread': -1, 'silent': True,
    'num_leaves': 2**4, 'learning_rate': 0.05, 'max_depth': 4,
    'max_bin': 255, 'subsample_for_bin': 50000,
    'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.6, 'reg_alpha': 1, 'reg_lambda': 0,
    'min_split_gain': 0.5, 'min_child_weight': 1, 'min_child_samples': 10, 'scale_pos_weight': 1}

# they can be used directly to build a LGBMClassifier (which is wrapped in a sklearn fashion)

ligh_mdl=lgb.LGBMClassifier(**t4_params)

ligh_mdl.fit(train_X, train_y)

print(metrics.f1_score(test_y,ligh_mdl.predict(test_X),average='weighted'))
# ligh_mdl.fit(train_pro_data[X], train_pro_data[Y])
pred_lgb=ligh_mdl.predict(test_pro_data[X])
print(np.unique(pred_lgb,return_counts=True))

pred_lgb=lbl.inverse_transform(pred_lgb.astype(int))
print(np.unique(pred_lgb))

sub_samp3=pd.DataFrame(test_pro_data['roadId'])
sub_samp3['noOfLanes']=pred_lgb

sub_samp3.to_csv(path+'1lgb_model_less_feature.csv',index=False)